# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 55 new papers today
          19 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/18 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.05679


extracting tarball to tmp_2210.05679...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.05684


extracting tarball to tmp_2210.05684...

...

extracting tarball to tmp_2210.06338...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.06390


extracting tarball to tmp_2210.06390...

 done.


/tmp/ipykernel_2295/4030337529.py:34: LatexWarning: 2210.06390 did not run properly
sequence item 1: expected str instance, TexNode found
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.05695-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.05695) | **The Jet Opening Angle and Event Rate Distributions of Short Gamma-ray  Bursts from Late-time X-ray Afterglows**  |
|| Alicia Rouco Escorial, et al. -- incl., <mark>Kerry Paterson</mark> |
|*Appeared on*| *2022-10-13*|
|*Comments*| *32 pages, 8 figures, 4 tables (Submitted to ApJ)*|
|**Abstract**| We present a comprehensive study of 29 short gamma-ray bursts (SGRBs) observed $\approx 0.8-60$ days post-burst using $Chandra$ and $XMM-Newton$. We provide the inferred distributions of SGRB jet opening angles and true event rates to compare against neutron star merger rates. We perform uniform analysis and modeling of their afterglows, obtaining 10 opening angle measurements and 19 lower limits. We report on two new opening angle measurements (SGRBs 050724A and 200411A) and eight updated values, obtaining a median value of $\langle \theta_{\rm j} \rangle \approx 6.1^{\circ}$ [-3.2$^{\circ}$,+9.3$^{\circ}$] (68\% confidence on the full distribution) from jet measurements alone. For the remaining events, we infer $\theta_{\rm j}\gtrsim 0.5-26^{\circ}$. We uncover a population of SGRBs with wider jets of $\theta_{\rm j} \gtrsim 10^{\circ}$ (including two measurements of $\theta_{\rm j} \gtrsim 15^{\circ}$), representing $\sim 28\%$ of our sample. Coupled with multi-wavelength afterglow information, we derive a total true energy of $\langle E_{\rm true, tot} \rangle \approx 10^{49}-10^{50}$\,erg which is consistent with MHD jet launching mechanisms. Furthermore, we determine a range for the beaming-corrected event rate of $\mathfrak{R}_{\rm true} \approx360-1800$ Gpc$^{-3}$ yr$^{-1}$, set by the inclusion of a population of wide jets on the low end, and the jet measurements alone on the high end. From a comparison with the latest merger rates, our results are consistent with the majority of SGRBs originating from binary neutron star mergers. However, our inferred rates are well above the latest neutron star-black hole merger rates, consistent with at most a small fraction of SGRBs originating from such mergers. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.05679-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.05679) | **Ironing the folds: The phase space chevrons of a GSE-like merger as a  dark matter subhalo detector**  |
|| <mark>Elliot Y. Davies</mark>, et al. |
|*Appeared on*| *2022-10-13*|
|*Comments*| *18 pages, 20 figures, submitted to MNRAS*|
|**Abstract**| Recent work uncovered features in the phase space of the Milky Way's stellar halo which may be attributed to the last major merger. When stellar material from a satellite is accreted onto its host, it phase mixes and appears finely substructured in phase space. For a high-eccentricity merger, this substructure most clearly manifests as numerous wrapping chevrons in $(v_r, r)$ space, corresponding to stripes in $(E, \theta_r)$ space. We introduce the idea of using this substructure as an alternative subhalo detector to cold stellar streams. We simulate an N-body merger akin to the GSE and assess the impact of subhaloes on these chevrons. We examine how their deformation depends on the mass, pericentre, and number of subhaloes. To quantify the impact of perturbers, we utilise the appearance of chevrons in $(E, \theta_r)$ space to introduce a new quantity -- the ironing parameter. We show that: (1) a single flyby of a massive ($\sim 10^{10}$ M$_{\odot}$) subhalo with pericentre comparable to, or within, the shell's apocentre smooths out the substructure, (2) a single flyby of a low mass ($\lesssim 10^8$ M$_{\odot}$) has negligible effect, (3) multiple flybys of subhalos derived from a subhalo mass function between $10^7-10^{10}$ M$_{\odot}$ cause significant damage if deep within the potential, (4) the effects of known perturbers (e.g. Sagittarius) should be detectable and offer constraints on their initial mass. The sensitivity to the populations of subhaloes suggests that we should be able to place an upper limit on the Milky Way's subhalo mass function. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.05684-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.05684) | **BICEP / Keck XVI: Characterizing Dust Polarization Through Correlations  with Neutral Hydrogen**  |
|| BICEP/Keck Collaboration, et al. -- incl., <mark>S. Harrison</mark>, <mark>S. Henderson</mark> |
|*Appeared on*| *2022-10-13*|
|*Comments*| *26 pages, 11 figures*|
|**Abstract**| We characterize Galactic dust filaments by correlating BICEP/Keck and Planck data with polarization templates based on neutral hydrogen (HI) observations. Dust polarization is important for both our understanding of astrophysical processes in the interstellar medium (ISM) and the search for primordial gravitational waves in the cosmic microwave background (CMB). In the diffuse ISM, HI is strongly correlated with the dust and partly organized into filaments that are aligned with the local magnetic field. We analyze the deep BICEP/Keck data at 95, 150, and 220 GHz, over the low-column-density region of sky where BICEP/Keck has set the best limits on inflationary theories. We separate the HI emission into distinct velocity components and demonstrate a detection of dust polarization correlated with the local Galactic HI but not with the HI associated with Magellanic Stream I. We present a robust, multi-frequency detection of polarized dust emission correlated with the filamentary HI morphology template down to 95 GHz. We find the correlation for frequencies between 95-353 GHz to be between ~10-50% over multipoles $20<\ell<200$. We measure the spectral index of the filamentary dust component spectral energy distribution to be $\beta = 1.54 \pm 0.13$. We find no evidence for decorrelation in this region between the filaments and the rest of the dust field or from the inclusion of dust associated with the intermediate velocity HI. Finally, we explore the morphological parameter space in the HI-based filamentary model. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.06219-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.06219) | **Multiple Regions of Nonthermal Quasi-Periodic Pulsations during the  Impulsive Phase of a Solar Flare**  |
|| Yingjie Luo, et al. -- incl., <mark>Rohit Sharma</mark> |
|*Appeared on*| *2022-10-13*|
|*Comments*| *19 pages, 12 figures, accepted for publication in The Astrophysical Journal*|
|**Abstract**| Flare-associated quasi-periodic pulsations (QPPs) in radio and X-ray wavelengths, particularly those related to nonthermal electrons, contain important information about the energy release and transport processes during flares. However, the paucity of spatially resolved observations of such QPPs with a fast time cadence has been an obstacle for us to further understand their physical nature. Here, we report observations of such a QPP event occurred during the impulsive phase of a C1.8-class eruptive solar flare using radio imaging spectroscopy data from the Karl G. Jansky Very Large Array (VLA) and complementary X-ray imaging and spectroscopy data. The radio QPPs, observed by the VLA in the 1--2 GHz with a sub-second cadence, are shown as three spatially distinct sources with different physical characteristics. Two radio sources are located near the conjugate footpoints of the erupting magnetic flux rope with opposite senses of polarization. One of the sources displays a QPP behavior with a ~5-s period. The third radio source, located at the top of the post-flare arcade, coincides with the location of an X-ray source and shares a similar period of ~25--45 s. We show that the two oppositely polarized radio sources are likely due to coherent electron cyclotron maser (ECM) emission. On the other hand, the looptop QPP source, observed in both radio and X-rays, is consistent with incoherent gyrosynchrotron and bremsstrahlung emission, respectively. We conclude that the concurrent, but spatially distinct QPP sources must involve multiple mechanisms which operate in different magnetic loop systems and at different periods. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.06251-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.06251) | **The evolution of HCO$^{+}$ in molecular clouds using a novel chemical  post-processing algorithm**  |
|| M. Panessa, et al. -- incl., <mark>L. Neumann</mark> |
|*Appeared on*| *2022-10-13*|
|*Comments*| *23 pages including appendix, 20 figures, submitted to MNRAS, comments are welcome*|
|**Abstract**| Modeling the internal chemistry of molecular clouds is critical to accurately simulating their evolution. To reduce computational expense, 3D simulations generally restrict their chemical modeling to species with strong heating and cooling effects. We address this by post-processing tracer particles in the SILCC-Zoom molecular cloud simulations. Using a chemical network of 39 species and 299 reactions (including freeze-out of CO and H$_2$O), and a novel iterative algorithm to reconstruct a filled density grid from sparse tracer particle data, we produce time-dependent density distributions for various species. We focus upon the evolution of HCO$^+$, which is a critical formation reactant of CO but is not typically modeled on-the-fly. We analyse the evolution of the tracer particles to assess the regime in which HCO$^+$ production preferentially takes place. We find that the HCO$^+$ content of the cold molecular gas forms in situ around $n_\textrm{HCO$^+$}\simeq10^3$-$10^4$ cm$^{-3}$, over a time-scale of approximately 1 Myr, rather than being distributed to this density regime via turbulent mixing from deeper in the cloud. We further show that the dominant HCO$^+$ formation pathway is dependent on the visual extinction, with the reaction H$_3^+$ + CO contributing 90% of the total HCO$^+$ production flux above $A_\textrm{V,3D}=3$. Using our novel grid reconstruction algorithm, we produce the very first maps of the HCO$^+$ column density, $N$(HCO$^+$), and show that it reaches values as high as $10^{15}$ cm$^{-2}$. We find that 50% of the HCO$^+$ mass is located in an $A_\textrm{V}$-range of $\sim$10-30, and in a density range of $10^{3.5}$-$10^{4.5}$ cm$^{-3}$. Finally, we compare our $N$(HCO$^+$) maps to recent observations of W49A and find good agreement. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.06338-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.06338) | **Lorentz invariance violation induced threshold anomaly versus very-high  energy cosmic photon emission from GRB20221009A**  |
|| <mark>Hao Li</mark>, Bo-Qiang Ma |
|*Appeared on*| *2022-10-13*|
|*Comments*| *4 latex pages, 2 figures*|
|**Abstract**| It is reported that the Large High Altitude Air Shower Observatory (LHAASO) observed very high energy photons from GRB 20221009A, with the highest energy reaching 18 TeV. We find that observation of such high energy photons is a quite nontrivial fact since extragalactic background light could absorb these photons severely and the flux is too weak to be observed. Therefore we discuss the potential new mechanism for us to observe these photons, and suggest that Lorentz invariance violation induced threshold anomaly of the process (\gamma\gamma\to e^-e^+) provides a candidate to explain this phenomenon. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.06283-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.06283) | **Use of Superfluid Helium to Observe Directionality of Galactic Dark  Matter**  |
|| <mark>George M. Seidel</mark>, Christian Enss |
|*Appeared on*| *2022-10-13*|
|*Comments*| *4 pages, 0 figures*|
|**Abstract**| The quasiparticle propagation away from the track of a highly ionizing particle in superfluid helium at low temperatures has previously been shown to exhibit anisotropy. We discuss the mechanism responsible for this behavior and show that it occurs for nuclear scattering by dark matter for recoil energies down to a few keV, and perhaps lower. This makes it possible to extend WIMP searches with interaction cross sections that reach into the neutrino floor in a meaningful energy range. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.06390-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.06390) | **Spectral cube extraction for the VLT/SPHERE IFS: Open-source pipeline  with full forward modeling and improved sensitivity**  |
|| <mark>Matthias Samland</mark>, et al. |
|*Appeared on*| *2022-10-13*|
|*Comments*| *17 pages, 11 figures. Software available at: this https URL*|
|**Abstract**| We present a new open-source data-reduction pipeline to reconstruct spectral data cubes from raw SPHERE integral-field spectrograph (IFS) data. The pipeline is written in Python and based on the pipeline that was developed for the CHARIS IFS. It introduces several improvements to SPHERE data analysis that ultimately produce significant improvements in postprocessing sensitivity. We first used new data to measure SPHERE lenslet point spread functions (PSFs) at the four laser calibration wavelengths. These lenslet PSFs enabled us to forward-model SPHERE data, to extract spectra using a least-squares fit, and to remove spectral crosstalk using the measured lenslet PSFs. Our approach also reduces the number of required interpolations, both spectral and spatial, and can preserve the original hexagonal lenslet geometry in the SPHERE IFS. In the case of least-squares extraction, no interpolation of the data is performed. We demonstrate this new pipeline on the directly imaged exoplanet 51 Eri b and on observations of the hot white dwarf companion to HD 2133. The extracted spectrum of HD 2133B matches theoretical models, demonstrating spectrophotometric calibration that is good to a few percent. Postprocessing on two 51 Eri b data sets demonstrates a median improvement in sensitivity of 80% and 30% for the 2015 and 2017 data, respectively, compared to the use of cubes reconstructed by the SPHERE Data Center. The largest improvements are seen for poorer observing conditions. The new SPHERE pipeline takes less than three minutes to produce a data cube on a modern laptop, making it practical to reprocess all SPHERE IFS data. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error sequence item 1: expected str instance, TexNode found</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2210.05695.md
    + _build/html/tmp_2210.05695/./TrueRate_paper.png
    + _build/html/tmp_2210.05695/./rate_CDF_paper.png
    + _build/html/tmp_2210.05695/./ALL_flux_LC.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\raf}[1]{\textcolor{orange}{#1}}$
$\newcommand{\NU}{\affiliation{Center for Interdisciplinary Exploration and Research in Astrophysics (CIERA) and Department of Physics and Astronomy, Northwestern University, 1800 Sherman Ave, Evanston, IL 60201, USA}}$
$\newcommand{\GSFC}{\affiliation{NASA Goddard Space Flight Center, University of Maryland, Baltimore County, Greenbelt, MD 20771, USA}}$
$\newcommand{\CfA}{\affiliation{Center for Astrophysics\:|\:Harvard \& Smithsonian, 60 Garden St. Cambridge, MA 02138, USA}}$
$\newcommand{\Einstein}{\altaffiliation{NASA Einstein Fellow}}$
$\newcommand{\NASA}{\altaffiliation{NASA Postdoctoral Fellow}}$
$\newcommand{\UAH}{\affiliation{Center for Space Plasma and Aeronomic Research, University of Alabama in Huntsville, 320 Sparkman Drive, Huntsville, AL 35899, USA}}$
$\newcommand{\USRA}{\affiliation{Science and Technology Institute, Universities Space Research Association, Huntsville, AL 35805, USA}}$
$\newcommand{\Arizona}{\affiliation{University of Arizona, Steward Observatory, 933 N. Cherry Avenue, Tucson, AZ 85721, USA}}$
$\newcommand{\Bath}{\affiliation{Department of Physics, University of Bath, Claverton Down, Bath, BA2 7AY, UK}}$
$\newcommand{\OU}{\affiliation{Astrophysical Institute, Department of Physics and Astronomy, 251B Clippinger Lab, Ohio University, Athens, OH 45701, USA}}$
$\newcommand{\Adler}{\affiliation{The Adler Planetarium, Chicago, IL 60605, USA}}$
$\newcommand{\GeminiN}{\affiliation{Gemini Observatory/NSF's NOIRLab, 670 N. A'ohoku Place, Hilo, HI, 96720, USA}}$
$\newcommand{\UMD}{\affiliation{Joint Space-Science Institute, University of Maryland, College Park, MD 20742, USA}}$
$\newcommand{\GWU}{\affiliation{Department of Physics, The George Washington University, Washington, DC 20052, USA}}$
$\newcommand{\Leicester}{\affiliation{School of Physics and Astronomy, University of Leicester, University Road, Leicester, LE1 7RH, UK}}$
$\newcommand{\Marin}{\affiliation{College of Marin, 120 Kent Avenue, Kentfield 94904 CA, USA}}$
$\newcommand{\UVI}{\affiliation{University of the Virgin Islands, \#2 Brewers bay road, Charlotte Amalie, 00802 USVI, USA}}$
$\newcommand{\Radboud}{\affiliation{Department of Astrophysics/IMAPP, Radboud University, 6525 AJ Nijmegen, The Netherlands}}$
$\newcommand{\Warwick}{\affiliation{Department of Physics, University of Warwick, Coventry, CV4 7AL, UK}}$
$\newcommand{\Birmingham}{\affiliation{Birmingham Institute for Gravitational Wave Astronomy and School of Physics and Astronomy, University of Birmingham, Birmingham B15 2TT, UK}}$
$\newcommand{\Edinburgh}{\affiliation{Institute for Astronomy, University of Edinburgh, Royal Observatory, Blackford Hill, EH9 3HJ, UK}}$
$\newcommand{\Caltech}{\affiliation{Cahill Center for Astrophysics, California Institute of Technology, 1200 E. California Blvd. Pasadena, CA 91125, USA}}$
$\newcommand{\LJMU}{\affiliation{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}}$
$\newcommand{\CCA}{\affiliation{Center for Computational Astrophysics, Flatiron Institute, 162 W. 5th Avenue, New York, NY 10011, USA}}$
$\newcommand{\Columbia}{\affiliation{Department of Physics and Columbia Astrophysics Laboratory, Columbia University, New York, NY 10027, USA}}$
$\newcommand{\CRESST}{\affiliation{Center for Research and Exploration in Space Science and Technology (CRESST) and NASA Goddard Space Flight Center, Greenbelt, MD 20771, USA}}$
$\newcommand{\Maryland}{\affiliation{Department of Physics, University of Maryland, Baltimore County, 1000 Hilltop Circle, Baltimore, MD 21250, USA}}$
$\newcommand{\MPIA}{\affiliation{Max-Planck-Institut f\"ur Astronomie (MPIA), Königstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{\Berkely}{\affiliation{Department of Astronomy, University of California, Berkeley, CA 94720-3411, USA}}$
$\newcommand{\IU}{\affiliation{Department of Astronomy, Indiana University, Bloomington, IN 47405-7105, USA}}$
$\newcommand{\Cornell}{\affiliation{Department of Astronomy, Cornell University, 404 Space Sciences Building, Ithaca, NY 14853, USA}}$
$\newcommand{\UChicago}{\affiliation{University of Chicago, 5801 S Ellis Ave, Chicago, IL 60637, USA}}$
$\newcommand{\Utah}{\affiliation{Department of Physics and Astronomy, University of Utah, James Fletcher Building, Salt Lake City, UT 84112,USA}}$
$\newcommand{\}{ra}$
$\newcommand{\}{dec}$
$\newcommand{\nod}{\nodata}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\raf$}[1]{\textcolor{orange}{#1}}$
$\newcommand{$\NU$}{\affiliation{Center for Interdisciplinary Exploration and Research in Astrophysics (CIERA) and Department of Physics and Astronomy, Northwestern University, 1800 Sherman Ave, Evanston, IL 60201, USA}}$
$\newcommand{$\GSFC$}{\affiliation{NASA Goddard Space Flight Center, University of Maryland, Baltimore County, Greenbelt, MD 20771, USA}}$
$\newcommand{$\CfA$}{\affiliation{Center for Astrophysics\:|\:Harvard \& Smithsonian, 60 Garden St. Cambridge, MA 02138, USA}}$
$\newcommand{$\Einstein$}{\altaffiliation{NASA Einstein Fellow}}$
$\newcommand{$\NASA$}{\altaffiliation{NASA Postdoctoral Fellow}}$
$\newcommand{$\UAH$}{\affiliation{Center for Space Plasma and Aeronomic Research, University of Alabama in Huntsville, 320 Sparkman Drive, Huntsville, AL 35899, USA}}$
$\newcommand{$\USRA$}{\affiliation{Science and Technology Institute, Universities Space Research Association, Huntsville, AL 35805, USA}}$
$\newcommand{$\Arizona$}{\affiliation{University of Arizona, Steward Observatory, 933 N. Cherry Avenue, Tucson, AZ 85721, USA}}$
$\newcommand{$\Bath$}{\affiliation{Department of Physics, University of Bath, Claverton Down, Bath, BA2 7AY, UK}}$
$\newcommand{$\OU$}{\affiliation{Astrophysical Institute, Department of Physics and Astronomy, 251B Clippinger Lab, Ohio University, Athens, OH 45701, USA}}$
$\newcommand{$\Adler$}{\affiliation{The Adler Planetarium, Chicago, IL 60605, USA}}$
$\newcommand{$\GeminiN$}{\affiliation{Gemini Observatory/NSF's NOIRLab, 670 N. A'ohoku Place, Hilo, HI, 96720, USA}}$
$\newcommand{$\UMD$}{\affiliation{Joint Space-Science Institute, University of Maryland, College Park, MD 20742, USA}}$
$\newcommand{$\GWU$}{\affiliation{Department of Physics, The George Washington University, Washington, DC 20052, USA}}$
$\newcommand{$\Leicester$}{\affiliation{School of Physics and Astronomy, University of Leicester, University Road, Leicester, LE1 7RH, UK}}$
$\newcommand{$\Marin$}{\affiliation{College of Marin, 120 Kent Avenue, Kentfield 94904 CA, USA}}$
$\newcommand{$\UVI$}{\affiliation{University of the Virgin Islands, \#2 Brewers bay road, Charlotte Amalie, 00802 USVI, USA}}$
$\newcommand{$\Radboud$}{\affiliation{Department of Astrophysics/IMAPP, Radboud University, 6525 AJ Nijmegen, The Netherlands}}$
$\newcommand{$\Warwick$}{\affiliation{Department of Physics, University of Warwick, Coventry, CV4 7AL, UK}}$
$\newcommand{$\Birmingham$}{\affiliation{Birmingham Institute for Gravitational Wave Astronomy and School of Physics and Astronomy, University of Birmingham, Birmingham B15 2TT, UK}}$
$\newcommand{$\Edinburgh$}{\affiliation{Institute for Astronomy, University of Edinburgh, Royal Observatory, Blackford Hill, EH9 3HJ, UK}}$
$\newcommand{$\Caltech$}{\affiliation{Cahill Center for Astrophysics, California Institute of Technology, 1200 E. California Blvd. Pasadena, CA 91125, USA}}$
$\newcommand{$\LJMU$}{\affiliation{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}}$
$\newcommand{$\CCA$}{\affiliation{Center for Computational Astrophysics, Flatiron Institute, 162 W. 5th Avenue, New York, NY 10011, USA}}$
$\newcommand{$\Columbia$}{\affiliation{Department of Physics and Columbia Astrophysics Laboratory, Columbia University, New York, NY 10027, USA}}$
$\newcommand{$\CRESST$}{\affiliation{Center for Research and Exploration in Space Science and Technology (CRESST) and NASA Goddard Space Flight Center, Greenbelt, MD 20771, USA}}$
$\newcommand{$\Maryland$}{\affiliation{Department of Physics, University of Maryland, Baltimore County, 1000 Hilltop Circle, Baltimore, MD 21250, USA}}$
$\newcommand{$\MPIA$}{\affiliation{Max-Planck-Institut f\"ur Astronomie (MPIA), Königstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{$\Berkely$}{\affiliation{Department of Astronomy, University of California, Berkeley, CA 94720-3411, USA}}$
$\newcommand{$\IU$}{\affiliation{Department of Astronomy, Indiana University, Bloomington, IN 47405-7105, USA}}$
$\newcommand{$\Cornell$}{\affiliation{Department of Astronomy, Cornell University, 404 Space Sciences Building, Ithaca, NY 14853, USA}}$
$\newcommand{$\UChicago$}{\affiliation{University of Chicago, 5801 S Ellis Ave, Chicago, IL 60637, USA}}$
$\newcommand{$\Utah$}{\affiliation{Department of Physics and Astronomy, University of Utah, James Fletcher Building, Salt Lake City, UT 84112,USA}}$
$\newcommand{\}{ra}$
$\newcommand{\}{dec}$
$\newcommand{$\nod$}{$\nod$ata}$</div>



<div id="title">

# The Jet Opening Angle and Event Rate Distributions of Short Gamma-ray Bursts from Late-time X-ray Afterglows

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2210.05695-b31b1b.svg)](https://arxiv.org/abs/2210.05695)<mark>Appeared on: 2022-10-13</mark> - _32 pages, 8 figures, 4 tables (Submitted to ApJ)_

</div>
<div id="authors">

A.~Rouco~Escorial, et al. -- incl., <mark>K.~Paterson</mark>

</div>
<div id="abstract">

**Abstract:** We present a comprehensive study of 29 short gamma-ray bursts (SGRBs) observed$\approx 0.8-60$days post-burst using\textit{Chandra}and\textit{XMM-Newton}. We provide the inferred distributions of SGRB jet opening angles and true event rates to compare against neutron star merger rates. We perform uniform analysis and modeling of their afterglows, obtaining 10 opening angle measurements and 19 lower limits. We report on two new opening angle measurements (SGRBs 050724A and 200411A) and eight updated values, obtaining a median value of$\langle \theta_{\rm j} \rangle \approx 6.1^{\circ}$[-3.2$^{\circ}$,+9.3$^{\circ}$](68\%confidence on the full distribution) from jet measurements alone. For the remaining events, we infer$\theta_{\rm j}\gtrsim 0.5-26^{\circ}$. We uncover a population of SGRBs with wider jets of$\theta_{\rm j} \gtrsim 10^{\circ}$(including two measurements of$\theta_{\rm j} \gtrsim 15^{\circ}$), representing$\sim 28\%$of our sample. Coupled with multi-wavelength afterglow information, we derive a total true energy of$\langle E_{\rm true, tot} \rangle \approx 10^{49}-10^{50}$erg which is consistent with MHD jet launching mechanisms. Furthermore, we determine a range for the beaming-corrected event rate of$\mathfrak{R}_{\rm true} \approx360-1800$Gpc$^{-3}$yr$^{-1}$, set by the inclusion of a population of wide jets on the low end, and the jet measurements alone on the high end. From a comparison with the latest merger rates, our results are consistent with the majority of SGRBs originating from binary neutron star mergers. However, our inferred rates are well above the latest neutron star-black hole merger rates, consistent with at most a small fraction of SGRBs originating from such mergers.

</div>

<div id="div_fig1">

<img src="tmp_2210.05695/./TrueRate_paper.png" alt="Fig7" width="100%"/>

**Figure 7. -** \textit{upper panel:} The SGRB true event rate distributions ($\mathfrak{R}_{\rm true}$) derived from the opening angle measurements, $\langle \mathfrak{R}_{\rm true, meas} \rangle = 1789[-1510,+6334]$ Gpc$^{-3}$ yr$^{-1}$, (orange) and mock, $\langle \mathfrak{R}_{\rm true, mock} \rangle = 362[-217,+4376]$ Gpc$^{-3}$ yr$^{-1}$, (yellow) samples using the observed rate of $\mathfrak{R}_{\rm obs}=10$ Gpc$^{-3}$ yr$^{-1}$. Median values of each distribution are indicated by black vertical lines. \textit{bottom panel:} We present a more realistic range of values of $\mathfrak{R}_{\rm true}$ that extends between the median values of the true event rate distributions derived from the opening angle measurements (vertical black dashed line) and mock samples (vertical grey dashed line). Other published rates are also shown as comparison. In particular, these rates are derived from: the detection of gravitational waves generated by BNS \citep{Abbott2020b} and NS-BH mergers \citep{Abbott2021}, SGRBs \citep{Fong2015a,Jin2018,Dichiara2020}, Galactic BNS \citep{OShaughnessy2010,Pol2020,Grunthal2021}, and the estimates for the BNS and NS-BH merger rates derived from population synthesis simulations by \citet{Sarin2022}. 68\% confidence levels are represented. (*fig:True_Rate*)

</div>
<div id="div_fig2">

<img src="tmp_2210.05695/./rate_CDF_paper.png" alt="Fig2" width="100%"/>

**Figure 2. -** The CDFs of true event rates for the opening angle measurement sample (orange solid line) and the mock sample including wide jets (yellow dashed line) assuming an observed local rate of $\mathfrak{R}_{\rm obs} \approx 10$ Gpc$^{-3}$ yr$^{-1}$. Comparing both distributions, we see that a larger number of wider jet opening angles in our sample would lead to a more constrained true event rate median value. (*fig:true_event_rate*)

</div>
<div id="div_fig3">

<img src="tmp_2210.05695/./ALL_flux_LC.png" alt="Fig4" width="100%"/>

**Figure 4. -** Unabsorbed X-ray flux ($0.3-10$ keV)  afterglow light curves of the 29 SGRBs in our sample. \textit{Swift} observations are represented as follows: WT-mode data with light grey thin diamond, and PC-mode data with dark grey circles. \textit{Chandra} and \textit{XMM-Newton} detections are depicted with dark blue squares and pink diamonds, respectively. The X-ray flux upper limits (3$\sigma$) for both observatories are shown with light blue and light pink triangles, respectively. (*fig:latetime_observations*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

62  publications in the last 7 days.
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers